In [1]:
import numpy as np
from utilities.Dataset import *
import pickle
import sys
from typing import List
import torch
import os

In [2]:
participant_data_path = './EEG/number'
processed_data_path = './EEG/processed_data'
split_data_path = './EEG/split_data'

In [3]:
ps = os.listdir(participant_data_path)
participants=[]
for p in ps:
    p= p[:3]
    if p not in participants:
        participants.append(p)
        
participants= sorted(participants)
# participants, len(participants)

In [4]:
load_name=f"userfold_data_scaled_p_dictionary-number"
data_dir = "./EEG/split_data/standard_scaled"
try:
    raw_user_fold= pickle.load(open(os.path.join(data_dir, f"{load_name}.pkl"), "rb"))
except:
    print(f"pickle file does not exist. Use EEG-Preprocess.ipynb and EEG-Split.ipynb to save data setting.")
    sys.exit()

In [5]:
from utilities.userfold_framework import *
from Models.AR_EEG_models import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List



In [ ]:
from torcheeg.models import EEGNet
import Models.model_func as Model_Func
from torch import nn
from Models.multi_models import *

DEVICE= torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

learning_rate = 0.00005
batch_size = 64
n_epochs = 300
transpose_channels=True
participants_dictionary=[]
# participants_online_dictionary=[]
participants_grads_dictionary={}
b_acc_list=[]
c0_acc_list=[]
c1_acc_list=[]

# EPOCH=[
    
# ]
for r in range(5):
    for i in range(len(participants)):

        train_dataloader, val_dataloader, classes, input_dim, class_ratio= user_fold_load(i,
                                                                                          raw_user_fold,
                                                                                          participants,
                                                                                          batch_size=batch_size,
                                                                                          transpose_channels=transpose_channels)

        eegnet= EEGNet(
            chunk_size=input_dim[1],
            num_electrodes=input_dim[0],
            num_classes=classes,
            kernel_1= 32,
            kernel_2=32,
            F1=8,
            F2=16,
            dropout=0.5
        ).to(DEVICE)

        classifier= LSTM_EEGNet_Wrapper(DEVICE, eegnet, input_dim).to(DEVICE)
    #     optimizer= torch.optim.RMSprop(classifier.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(classifier.parameters(), lr = learning_rate)


        criterion= torch.nn.CrossEntropyLoss(weight=torch.tensor(class_ratio, dtype=torch.float).to(DEVICE))
    #     criterion = nn.NLLLoss(weight=torch.tensor(class_ratio, dtype=torch.float).to(DEVICE))

        scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)

        saved_dir= f"./EEG/saved_models/Userfold/run{r}"
        model= EEGNet_IE_EEG_Wrapper(
            DEVICE, classifier, input_dim[0]
        ).to(DEVICE)

        train_func= eeg_grad_train
        model.training_procedure(iteration=n_epochs,
                                 train_dataloader=train_dataloader,
                                 val_dataloader=val_dataloader,
                                 print_cycle=2,
                                 path=f"./dictionary/intermdiate_dicts",
                                 loss_func=criterion,
                                 optimiser=optimizer, #scheduler=scheduler,
                                 train_func=train_func
                                )
        if model.epoch == n_epochs+1:
            EPOCH= n_epochs
        else:
            EPOCH= model.epoch

        torch.save(model.state_dict(), 
               os.path.join(
                   saved_dir, f"Userfold-{participants[i]}-LSTM_EEGNet-Grad_EEG-e{EPOCH}.pt"
               )
        )
        pickle.dump(model.return_IE_grad(),
                    open(
                        os.path.join(
                            saved_dir, f"Userfold-{participants[i]}-LSTM_EEGNet-Grad_EEG-grad_all-e{EPOCH}.pkl"
                        ), "wb")
        )
    # OR
    #     model.load_state_dict(
    #     torch.load(
    #         open(
    #             os.path.join(
    #                 saved_dir, f"Userfold-{participants[i]}-EEGNet-IE_Multivariate-e{n_epochs}.pt"
    #             ), "rb"
    #         )
    #               )
    #     )
    #     model.IE_grad= pickle.load(
    #             open(
    #                 os.path.join(
    #                     saved_dir, f"Userfold-{participants[i]}-EEGNet-IE_Multivariate-grad_all-e{n_epochs}.pkl"
    #                 ),"rb"
    #             )
    #         )


        prediction, dictionary= model.prediction_procedure(val_dataloader, dict_flag=True)

        ys= np.concatenate([y.detach().cpu().numpy() for x, y in val_dataloader])

        c0_acc, c1_acc, b_acc= calculate_accuracy(ys, prediction)
        print("c0_acc", c0_acc, ", c1_acc", c1_acc, ", b_acc", b_acc)
        b_acc_list.append(b_acc)
        c0_acc_list.append(c0_acc)
        c1_acc_list.append(c1_acc)
    #     participants_grads_dictionary[participants[i]]= model.return_IE_grad()
        participants_dictionary.append(dictionary)
    
   

tmp=[]
for i, dictionary in enumerate(participants_dictionary):
    print(f"User {participants[i]} f1: {dictionary['weighted avg']['f1-score']} acc: {dictionary['accuracy']}")
    print(f" c0: {c0_acc_list[i]} c1: {c1_acc_list[i]} bacc: {b_acc_list[i]}")
    tmp.append(dictionary['weighted avg']['f1-score'])

print(f"average {np.mean(tmp)}")
print()
print(np.array(b_acc_list).mean())
print(np.array(c1_acc_list).mean())
print(np.array(c0_acc_list).mean())


Iterations:   0%|                                           | 0/300 [00:00<?, ?it/s]

Epoch  0 , loss 0.722334153511945


Iterations:   0%|                                   | 1/300 [00:02<14:17,  2.87s/it]

Epoch:  0
t_loss:  0.722334153511945 , v_loss:  0.6915584206581116
t_acc:  0.47307812013694367 , v_acc:  0.6894409937888198
t_recall:  0.49732572981430395 , v_recall:  0.5
t_prec:  0.49769847344474216 , v_prec:  0.3447204968944099
t_f:  0.46364953761590655 , v_f:  0.4080882352941176
////////


Iterations:   1%|▏                                  | 2/300 [00:03<08:17,  1.67s/it]

...

Epoch  167 , loss 0.39374087955437453
Epoch  168 , loss 0.3935321575286342


Iterations:  56%|██████████████████▌              | 169/300 [03:17<02:35,  1.19s/it]

Epoch:  168
t_loss:  0.3935321575286342 , v_loss:  0.7624349594116211
t_acc:  0.7160224438902744 , v_acc:  0.7289156626506024
t_recall:  0.5441808256014253 , v_recall:  0.5586080586080586
t_prec:  0.7340550015051612 , v_prec:  0.7153846153846153
t_f:  0.504850955713586 , v_f:  0.5362264853790277
////////


Iterations:  57%|██████████████████▋              | 170/300 [03:19<02:27,  1.14s/it]

Epoch  169 , loss 0.39783673660427915


In [ ]:
userfold_results_summary(participants_dictionary, participants)
userfold_classwise_results_summary(participants_dictionary, participants)


In [ ]:
participant_grad={"AUC":[],"ROC":[],"STD":[]}
for i in range(len(participants)):
    g= pickle.load(
            open(
                os.path.join(
                    saved_dir, f"Userfold-{participants[i]}-LSTM_EEGNet-Grad_EEG-grad_all-e{n_epochs}.pkl"
                ),"rb"
            )
    )
    
    participant_grad["AUC"].append(
        pd.DataFrame(
            Grad_AUC_with_grad(
                g, n_epochs, np.zeros(len(channel_names)), x_shape=0
            ).values()
        ).to_numpy().reshape(-1)
    )
    
    participant_grad["ROC"].append(
        pd.DataFrame(
            Grad_ROC_with_grad(
                g, n_epochs, np.zeros(len(channel_names)), x_shape=0
            ).values()
        ).to_numpy().reshape(-1)
    )
    
    participant_grad["STD"].append(
        pd.DataFrame(
            Grad_STD_with_grad(
                g, n_epochs, np.zeros(len(channel_names)), x_shape=0
            ).values()
        ).to_numpy().reshape(-1)
    )

In [ ]:
# channel_names=[i for i in range(input_dim[0])]
channel_names=["AFz","F3","F1","Fz","F2","F4","FC5","FC3","FC1","FCz","FC2",
               "FC4","FC6","C5","C3","C1","Cz","C2","C4","C6","CP5","CP3",
               "CP1","CPz","CP2","CP4","CP6","P3","P1","Pz","P2","P4"]

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import matplotlib
import mne
info= mne.create_info(channel_names, sfreq=500, ch_types=32*["eeg"])
info.set_montage("standard_1020")

fig, ax= plt.subplots(2,3)
methods=["AUC", "ROC", "STD"]
for idx, i in enumerate(methods):
    scaler= MinMaxScaler()
    scaled_avg_grad= scaler.fit_transform(np.array(participant_grad[i]).mean(0).reshape(-1,1))
#     scaled_avg_grad= avg_grad[i].sum(1)

    minmax_im, _= mne.viz.plot_topomap(
        scaled_avg_grad.reshape(-1),
        info,
        ch_type= "eeg",
        sensors=True,
        names=channel_names,
        cmap="Blues",
        axes=ax[0][idx],
        show=False,
        extrapolate="local"
        #,vlim=(0,1)
    #     sphere=0.09
    )
    ax[0][idx].set_title(f"MinMax: {i}")
    
for idx, i in enumerate(methods):
    scaler= RobustScaler()
    scaled_avg_grad= scaler.fit_transform(np.array(participant_grad[i]).mean(0).reshape(-1,1))
#     scaled_avg_grad= avg_grad[i].sum(1)

    robust_im, _= mne.viz.plot_topomap(
        scaled_avg_grad.reshape(-1),
        info,
        ch_type= "eeg",
        sensors=True,
        names=channel_names,
        cmap="Blues",
        axes=ax[1][idx],
        show=False,
        extrapolate="local",
#         vlim=(0,None)
    #     sphere=0.09
    )
    ax[1][idx].set_title(f"Robust: {i}")
    
fig.set_figheight(25)
fig.set_figwidth(25)
# fig.add_axes(ax)
cbar_ax= fig.add_axes([.92,0.66, 0.02,0.18])
clb= fig.colorbar(minmax_im, cax=cbar_ax)
clb.set_label("Importance Estimate", rotation=270,labelpad=20, fontsize=18)

cbar_ax= fig.add_axes([.92,0.22, 0.02,0.18])
clb= fig.colorbar(robust_im, cax=cbar_ax)
clb.set_label("Importance Estimate", rotation=270,labelpad=20, fontsize=18)

for tt in plt.findobj(fig, matplotlib.text.Text):
    if tt.get_text() in channel_names:
        tt.set_fontsize(14)
# plt.tight_layout()